# Lab 3: Agent Architectures

In this lab, we will explore 3 different agent architectures:

1. **Parallel Agent** (current setup) - All tools run concurrently
2. **Serial Tool Calling Agent** - Tools run sequentially in a fixed order
3. **Router-Based Agent** - Central router dynamically selects which tool to call next

We will trace the outputs of each agent for comparison and analyze:
- **Latencies** - Execution time differences between architectures
- **Output Quality** - Response content and completeness
- **Arize Tracing** - Observability and debugging insights

This comparison will help us understand the trade-offs between different agent orchestration patterns.


## Set Up

Make sure your Arize Space ID and Arize API key are set:

In [ ]:
import os
from getpass import getpass

os.environ["ARIZE_SPACE_ID"] = globals().get("ARIZE_SPACE_ID") or getpass("🔑 Enter your Arize Space ID: ")

os.environ["ARIZE_API_KEY"] = globals().get("ARIZE_API_KEY") or getpass("🔑 Enter your Arize API Key: ")

In [ ]:
from arize.otel import register
from openinference.instrumentation.openai import OpenAIInstrumentor

# configure the Arize tracer
tracer_provider = register(
    space_id=os.getenv("ARIZE_SPACE_ID"),
    api_key=os.getenv("ARIZE_API_KEY"),
    project_name="ai-trip-planner",
)
OpenAIInstrumentor().instrument(tracer_provider=tracer_provider)

In [ ]:
import time
from typing import Dict, Any
import requests
import pandas as pd

BASE_URL = "http://localhost:8000" #Modify if needed

print("✅ Setup complete!")


## Test Data

Let's define some test scenarios to compare architectures:


In [ ]:
TEST_SCENARIOS = [
    {
        "name": "Simple Trip",
        "destination": "Tokyo, Japan",
        "duration": "5 days",
        "budget": "moderate",
        "interests": "food, culture"
    },
    {
        "name": "Complex Trip",
        "destination": "Paris, France",
        "duration": "10 days",
        "budget": "luxury",
        "interests": "art, history, food, shopping"
    },
    {
        "name": "Budget Trip",
        "destination": "Bangkok, Thailand",
        "duration": "7 days",
        "budget": "budget",
        "interests": "temples, street food"
    }
]

print(f"✅ Defined {len(TEST_SCENARIOS)} test scenarios")

## Architecture 1: Current Parallel Architecture

The current implementation uses **parallel execution** where all agents (research, budget, local) run concurrently. This approach:

- **Maximizes Speed**: All agents execute simultaneously, reducing total execution time
- **Efficient Resource Usage**: Takes advantage of concurrent processing capabilities  
- **Independent Operations**: Each agent works on its specific domain without waiting for others



In [ ]:
def test_current_architecture(scenario: Dict[str, str]) -> Dict[str, Any]:
    """Test the current parallel architecture"""
    start_time = time.time()
    
    try:
        response = requests.post(
            f"{BASE_URL}/plan-trip",
            json=scenario,
            timeout=60
        )
        
        end_time = time.time()
        execution_time = end_time - start_time
        
        if response.status_code == 200:
            result = response.json()
            return {
                "architecture": "Parallel",
                "scenario": scenario["name"],
                "success": True,
                "execution_time": execution_time,
                "response_length": len(result.get("result", "")),
                "tool_calls": len(result.get("tool_calls", [])),
                "response": result
            }
        else:
            return {
                "architecture": "Parallel",
                "scenario": scenario["name"],
                "success": False,
                "execution_time": execution_time,
                "error": f"HTTP {response.status_code}: {response.text}"
            }
    except Exception as e:
        end_time = time.time()
        return {
            "architecture": "Parallel",
            "scenario": scenario["name"],
            "success": False,
            "execution_time": end_time - start_time,
            "error": str(e)
        }

print("🧪 Testing Current Parallel Architecture...")
parallel_results = []

for scenario in TEST_SCENARIOS:
    print(f"  Testing: {scenario['name']}")
    result = test_current_architecture(scenario)
    parallel_results.append(result)
    print(f"    ✅ Success: {result['success']}, Time: {result['execution_time']:.2f}s")

print(f"\n✅ Parallel architecture testing complete!")
print(f"Success rate: {sum(1 for r in parallel_results if r['success'])}/{len(parallel_results)}")


## Architecture 2: Serial Implementation
Let's implement a serial version that runs the same agent. In the serial architecture, the agent calls its tools one after another. Each tool handles a different aspect of the trip. This setup makes it easy to trace how information builds up over time, with each phase feeding into the next in a predictable sequence.


In [ ]:
def test_serial_architecture_sequential(scenario: Dict[str, str]) -> Dict[str, Any]:
    """Test serial architecture by running agents sequentially"""
    start_time = time.time()
    
    print(f"  🔍 Step 1: Research Agent for {scenario['name']}")
    research_start = time.time()
    
    research_request = {
        "destination": scenario["destination"],
        "duration": scenario["duration"],
        "budget": scenario["budget"],
        "interests": "research, essential info, weather, visa"
    }
    
    research_response = requests.post(
        f"{BASE_URL}/plan-trip",
        json=research_request,
        timeout=60
    )
    research_time = time.time() - research_start
    
    print(f"  💰 Step 2: Budget Agent for {scenario['name']}")
    budget_start = time.time()
    
    budget_request = {
        "destination": scenario["destination"],
        "duration": scenario["duration"],
        "budget": scenario["budget"],
        "interests": "budget, pricing, costs, expenses"
    }
    
    budget_response = requests.post(
        f"{BASE_URL}/plan-trip",
        json=budget_request,
        timeout=60
    )
    budget_time = time.time() - budget_start
    
    print(f"  🏘️ Step 3: Local Agent for {scenario['name']}")
    local_start = time.time()
    
    local_request = {
        "destination": scenario["destination"],
        "duration": scenario["duration"],
        "budget": scenario["budget"],
        "interests": scenario["interests"]  
    }
    
    local_response = requests.post(
        f"{BASE_URL}/plan-trip",
        json=local_request,
        timeout=60
    )
    local_time = time.time() - local_start
    
    print(f"  📋 Step 4: Final Synthesis for {scenario['name']}")
    synthesis_start = time.time()

    synthesis_request = {
        "destination": scenario["destination"],
        "duration": scenario["duration"],
        "budget": scenario["budget"],
        "interests": scenario["interests"],
        "previous_results": {
            "research": research_response.json(),
            "budget": budget_response.json(),
            "local": local_response.json()
        }
    }

    final_response = requests.post(f"{BASE_URL}/plan-trip", json=synthesis_request, timeout=60)
    synthesis_time = time.time() - synthesis_start
    final_result = final_response.json()

    end_time = time.time()
    total_execution_time = end_time - start_time

    combined_response = {
        "result": f"Serial itinerary for {scenario['destination']}: "
                  f"Sequential execution completed in {total_execution_time:.2f}s\n\n"
                  + final_result.get("result", ""),
        "phases": [
            {"phase": "research", "duration": f"{research_time:.2f}s", "order": 1},
            {"phase": "budget", "duration": f"{budget_time:.2f}s", "order": 2},
            {"phase": "local", "duration": f"{local_time:.2f}s", "order": 3},
            {"phase": "synthesis", "duration": f"{synthesis_time:.2f}s", "order": 4}
        ],
        "execution_details": {
            "total_time": total_execution_time,
            "research_time": research_time,
            "budget_time": budget_time,
            "local_time": local_time,
            "synthesis_time": synthesis_time
        }
    }

    return {
        "architecture": "Serial-Sequential",
        "scenario": scenario["name"],
        "success": True,
        "execution_time": total_execution_time,
        "response_length": len(combined_response["result"]),
        "response": combined_response
    }


In [ ]:
print("🧪 Testing Serial Architecture (Sequential Implementation)...")
serial_sequential_results = []

for scenario in TEST_SCENARIOS:
    print(f"  Testing: {scenario['name']}")
    result = test_serial_architecture_sequential(scenario)
    serial_sequential_results.append(result)
    print(f"    ✅ Success: {result['success']}, Time: {result['execution_time']:.2f}s")
    
    if result['success']:
        print(f"    📝 Response length: {result['response_length']} chars")
        print(f"    ⏱️ Total execution time: {result['execution_time']:.2f}s")

print(f"\n✅ Serial architecture testing complete!")
print(f"Success rate: {sum(1 for r in serial_sequential_results if r['success'])}/{len(serial_sequential_results)}")


## Architecture 3: Central Router with Dynamic Tool Selection

This code uses priority-based system to decide which tool to run next.

Here's how it works:

- Follows a priority list: A router checks for missing information in a specific order: budget, then research, then local.
- Calls tools one by one: It runs each tool (like a "budget" tool to find flight costs) by making an API call.
- Synthesizes a final plan: Once all prioritized tools have completed, it runs a final "synthesis" step to combine all the information into a single trip itinerary.
- Reports the results: Finally, it provides a report on the entire process, including the steps taken and the total time it took.

In short, it's a test of an automated workflow where a central "brain" directs the process based on simple rules to ensure all necessary steps are completed.

In [ ]:
def test_router_architecture(scenario: Dict[str, str]) -> Dict[str, Any]:
    """Test router architecture with simple priority-based dynamic tool selection"""
    start_time = time.time()

    collected_info = {
        "research": None,
        "budget": None,
        "local": None,
        "synthesis": None
    }

    tool_priority = ["budget", "research", "local"]

    def router_decision(current_info):
        """Decide the next tool to run based on priority and missing info"""
        for tool in tool_priority:
            if current_info[tool] is None:
                return tool, f"Running {tool} step"
        return "synthesis", "All tools complete, proceed to synthesis"

    # Routing loop
    max_iterations = len(tool_priority) + 1 
    iteration = 0
    routing_decisions = []

    print(f"🧠 Router starting for {scenario['name']}")

    while iteration < max_iterations:
        iteration += 1

        next_tool, reasoning = router_decision(collected_info)
        routing_decisions.append({
            "iteration": iteration,
            "decision": next_tool,
            "reasoning": reasoning,
            "context": f"Destination: {scenario['destination']}, Budget: {scenario['budget']}, Interests: {scenario['interests']}"
        })

        print(f"  🔄 Iteration {iteration}: Router selected {next_tool}")
        print(f"  💭 Reasoning: {reasoning}")

        if next_tool == "synthesis":
            print("  📋 Final synthesis")
            synthesis_start = time.time()

            synthesis_request = {
                "destination": scenario["destination"],
                "duration": scenario["duration"],
                "budget": scenario["budget"],
                "interests": scenario["interests"],
                "previous_results": collected_info 
            }

            synthesis_response = requests.post(
                f"{BASE_URL}/plan-trip",
                json=synthesis_request,
                timeout=60
            )
            synthesis_time = time.time() - synthesis_start

            collected_info["synthesis"] = synthesis_response.json()
            print(f"  ✅ Synthesis completed in {synthesis_time:.2f}s")
            break
        else:
            tool_start = time.time()
            tool_request = {
                "destination": scenario["destination"],
                "duration": scenario["duration"],
                "budget": scenario["budget"],
                "interests": next_tool  # focus = tool name
            }

            tool_response = requests.post(
                f"{BASE_URL}/plan-trip",
                json=tool_request,
                timeout=60
            )
            tool_time = time.time() - tool_start

            collected_info[next_tool] = tool_response.json()
            print(f"  ✅ {next_tool} completed in {tool_time:.2f}s")

    end_time = time.time()
    total_execution_time = end_time - start_time

    # Build response
    router_response = {
        "result": f"Priority-based Router itinerary for {scenario['destination']} completed in {total_execution_time:.2f}s",
        "routing_decisions": routing_decisions,
        "collected_info": {
            "research_available": collected_info["research"] is not None,
            "budget_available": collected_info["budget"] is not None,
            "local_available": collected_info["local"] is not None,
            "synthesis_available": collected_info["synthesis"] is not None
        },
        "execution_details": {
            "total_time": total_execution_time,
            "iterations": iteration,
            "routing_decisions": len(routing_decisions),
            "tool_calls": sum([
                len(collected_info[tool].get("tool_calls", []))
                for tool in ["research", "budget", "local", "synthesis"]
                if collected_info[tool] is not None
            ])
        }
    }

    return {
        "architecture": "Router-Priority",
        "scenario": scenario["name"],
        "success": True,
        "execution_time": total_execution_time,
        "response_length": len(router_response["result"]),
        "tool_calls": router_response["execution_details"]["tool_calls"],
        "response": router_response
    }

In [ ]:
results = []
for scenario in TEST_SCENARIOS:
    result = test_router_architecture(scenario)
    results.append(result)
    print(result)

# Results & Analysis

### 🚀 **Performance Comparison**
The results clearly demonstrate the **latency differences** between architectures:

- **Parallel Architecture**: Fastest execution time (all agents run concurrently)
- **Serial Architecture**: Slower execution (agents run sequentially) 
- **Router Architecture**: Moderate execution time (dynamic tool selection adds overhead)

**Parallelization significantly outperforms both serial execution and orchestrator patterns** in terms of raw speed.

### 📊 **What We Can See Immediately via Arize Tracing**
- **Execution Times**: Clear performance differences between architectures
- **Response Quality**: All architectures produce similar quality outputs
- **Tool Usage**: Different patterns of tool utilization
an see latency differences immediately, **Arize tracing provides much deeper insights**:

- **Tool Call Patterns**: See exactly which tools are called and when
- **Execution Flow**: Visualize the sequence of operations

### 🎯 **Architecture Trade-offs**
- **Parallel**: Best for speed, but harder to debug and trace
- **Serial**: Predictable execution, easy to trace, but slower
- **Router**: Intelligent and adaptive, but adds complexity

The choice of architecture depends on your specific requirements: speed vs. debuggability vs. intelligence.